# 5 - Analyses statistiques (proportion)

L'objectif ici est de reproduire toutes les analyses statistiques de base sur la république en utilisant les occurrences en proportion de la parole totale ou ici plus exactement la proportion d'énoncés mobilisant l'idée de république sur tous les énoncés. 
Pour cela on prend le fichier de base auquel on a ajouté la colonne avec le résultat de la regex en true/false. 

In [6]:
import pandas as pd
import plotly.express as px

In [11]:
df = pd.read_csv(
    "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS_République_FDM_proportion.csv", low_memory=False, dtype={"ID_orateur": str}
)

## Étape 1 : avoir une colonne proportion

## Analyses temporelles

In [12]:
import datetime
import locale

# Active la locale française (nécessaire pour le format)
locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

'fr_FR.UTF-8'

In [13]:
df["dateSeance_ts"] = pd.to_datetime(df["dateSeanceJour"], format="%A %d %B %Y")
df["dateSeance_day"] = df["dateSeance_ts"].dt.normalize()  

In [ ]:
# Rajouter pour avoir case insensible (= "république d'arménie" et pas que "République d'arménie")
pattern_pays = re.compile(pattern_pays, flags=re.IGNORECASE)

### Données corpus intégral

In [14]:
# Compter True/False par jour
df_daily = (
    df.groupby(df["dateSeance_day"].dt.date)["Republique_FDM_Regex"]
    .value_counts(normalize=True)  # calcule directement les proportions
    .rename("proportion")
    .reset_index()
)

# Garder uniquement les "True"
df_daily_true = df_daily[df_daily["Republique_FDM_Regex"] == True]

# aficher les 25 dates les plus fréquentes sous forme de tableau
table = df_daily_true.sort_values("proportion", ascending=False).head(20)
table

,dateSeance_day,Republique_FDM_Regex,proportion
4,2017-07-03,True,0.428571
1084,2021-02-01,True,0.335404
316,2018-07-09,True,0.264706
1908,2024-03-04,True,0.255319
1236,2021-06-28,True,0.248503
839,2020-03-22,True,0.235294
1092,2021-02-05,True,0.224684
1262,2021-07-23,True,0.167598
1377,2022-01-06,True,0.138728
1088,2021-02-03,True,0.137255


**Remarques**
*Le 3 juillet 2017, le 9 juillet 2018, le 4 mars 2024 sont parmis les 5 principales dates car parlement réuni en Congrès*

*le 1 février 2021, le 28 juin 2021, 5 février 2021, le 23 juillet 2021, le 3 février 2021, 30 juin 2021 (et 5 avril 2023 : bilan de la loi), 1er juillet 2021, le 12 février 2021, renvoient quant à eux à la discussion du projet de loi "confortant le respect des principes de la République". **==> Suivre en détail le processus législatif de ce projet de loi car moment central !!***
*On a aussi le 22 mars 2020, très courte séance (commencée à 18h30) sur l’urgence du covid et un hommage*

*6 janvier 2022, 6 juin 2022, 13 mars 2018 : réforme territoriale Nouvelle-Calédonie*

*25 janvier 2024, 8 juillet 2019 et 16 janvier 2020 : accords internationaux avec présence d'expressions comme "gouvernement de la République française", de pays sous forme adjectivable (ex : "république arménienne") ou avec république en miniscule --> moins présent maintenant que exclus*

*11 février 2019 sur "l'école de la confiance"*
*12 et 13 juillet 2018 sur le  projet de loi constitutionnelle pour une Démocratie plus représentative, responsable et efficace*

In [15]:
# Grouper par mois
df_monthly = (
    df.groupby(df["dateSeance_day"].dt.to_period("M"))["Republique_FDM_Regex"]
    .value_counts(normalize=True)
    .rename("proportion")
    .reset_index()
)

# Transformer la période en datetime (pour l'axe X)
df_monthly["dateSeance_day"] = df_monthly["dateSeance_day"].dt.to_timestamp()

# Garder True uniquement
df_monthly_true = df_monthly[df_monthly["Republique_FDM_Regex"] == True]

# Choisir une année
annee = 2021

# Filtrer le DataFrame sur l'année choisie
df_annee_mois = df_monthly_true[df_monthly_true["dateSeance_day"].dt.year == annee]

# Graphique
fig_annee_mois = px.bar(
    df_annee_mois,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par mois en 2021",
    labels={"proportion": "% des occurences mensualisées", "dateSeance_day": "Mois"},
    template="plotly_white",
)

fig_annee_mois.show()

In [16]:
# Compter True/False par jour
df_daily = (
    df.groupby(df["dateSeance_day"].dt.date)["Republique_FDM_Regex"]
    .value_counts(normalize=True)  # calcule directement les proportions
    .rename("proportion")
    .reset_index()
)

# Garder uniquement les "True"
df_daily_true = df_daily[df_daily["Republique_FDM_Regex"] == True]

# Graphique
fig_daily = px.line(
    df_daily_true,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par jour",
    labels={"proportion": "% des occurences", "dateSeance_day": "Date"},
    template="plotly_white",
)

fig_daily.show()


In [17]:
# Grouper par semaine
df_weekly = (
    df.groupby(df["dateSeance_day"].dt.to_period("W"))["Republique_FDM_Regex"]
    .value_counts(normalize=True)
    .rename("proportion")
    .reset_index()
)

# Transformer la période en datetime (pour l'axe X)
df_weekly["dateSeance_day"] = df_weekly["dateSeance_day"].dt.to_timestamp()

# Garder True uniquement
df_weekly_true = df_weekly[df_weekly["Republique_FDM_Regex"] == True]

# Graphique
fig_weekly = px.bar(
    df_weekly_true,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par semaine",
    labels={"proportion": "% des occurences hebdomaire", "dateSeance_day": "Semaine"},
    template="plotly_white",
)

fig_weekly.show()


In [18]:
# Grouper par mois
df_monthly = (
    df.groupby(df["dateSeance_day"].dt.to_period("M"))["Republique_FDM_Regex"]
    .value_counts(normalize=True)
    .rename("proportion")
    .reset_index()
)

# Transformer la période en datetime (pour l'axe X)
df_monthly["dateSeance_day"] = df_monthly["dateSeance_day"].dt.to_timestamp()

# Garder True uniquement
df_monthly_true = df_monthly[df_monthly["Republique_FDM_Regex"] == True]

# Graphique
fig_monthly = px.bar(
    df_monthly_true,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par mois",
    labels={"proportion": "% des occurences mensualisées", "dateSeance_day": "Mois"},
    template="plotly_white",
)

fig_monthly.show()


In [19]:
# Grouper par année
df_yearly = (
    df.groupby(df["dateSeance_day"].dt.year)["Republique_FDM_Regex"]
    .value_counts(normalize=True)
    .rename("proportion")
    .reset_index()
)

# Garder True uniquement
df_yearly_true = df_yearly[df_yearly["Republique_FDM_Regex"] == True]

# Graphique
fig_yearly = px.bar(
    df_yearly_true,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par année",
    labels={"proportion": "% des occurences annualisées", "dateSeance_day": "Année"},
    template="plotly_white",
)

fig_yearly.show()


### Données corpus annualisé 

In [ ]:
# Grouper par mois
df_monthly = (
    df.groupby(df["dateSeance_day"].dt.to_period("M"))["Republique_FDM_Regex"]
    .value_counts(normalize=True)
    .rename("proportion")
    .reset_index()
)

# Transformer la période en datetime (pour l'axe X)
df_monthly["dateSeance_day"] = df_monthly["dateSeance_day"].dt.to_timestamp()

# Garder True uniquement
df_monthly_true = df_monthly[df_monthly["Republique_FDM_Regex"] == True]

# Choisir une année
annee = 2021

# Filtrer le DataFrame sur l'année choisie
df_annee_mois = df_monthly_true[df_monthly_true["dateSeance_day"].dt.year == annee]

# Graphique
fig_annee_mois = px.bar(
    df_annee_mois,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par mois en 2021",
    labels={"proportion": "% des occurences mensualisées", "dateSeance_day": "Mois"},
    template="plotly_white",
)

fig_annee_mois.show()

In [24]:
# Grouper par semaine
df_weekly = (
    df.groupby(df["dateSeance_day"].dt.to_period("W"))["Republique_FDM_Regex"]
    .value_counts(normalize=True)
    .rename("proportion")
    .reset_index()
)

# Transformer la période en datetime (pour l'axe X)
df_weekly["dateSeance_day"] = df_weekly["dateSeance_day"].dt.to_timestamp()

# Garder True uniquement
df_weekly_true = df_weekly[df_weekly["Republique_FDM_Regex"] == True]

# Choisir une année
annee = 2024

# Filtrer le DataFrame sur l'année choisie
df_annee_semaine = df_weekly_true[df_weekly_true["dateSeance_day"].dt.year == annee]

# Graphique
fig_annee_semaine = px.bar(
    df_annee_semaine,
    x="dateSeance_day",
    y="proportion",
    title="Proportion occurrences de la 'République' par semaine en 2024",
    labels={"proportion": "% des occurences mensualisées", "dateSeance_day": "Mois"},
    template="plotly_white",
)

fig_annee_semaine.show()

### Par groupes (nécessite d'avoir fait la fusion avec les données de DATAN)

### Par personnel politique / individuellement

#### Les principaux orateurs sur la période / par législature

#### Ici en % des orateurs ayant mobilisé au moins 10 fois la "République"

Reprendre ici en essayant de comprendre exactement ce qui se fait

In [49]:
# Compter le nombre de fois où chaque orateur dit "République"
counts = (
    df.groupby("nom_orateur")["Republique_FDM_Regex"]
    .agg(total_mentions="count", true_mentions=lambda x: (x == True).sum())
    .reset_index()
)

# Calcul de la proportion
counts["proportion_true"] = counts["true_mentions"] / counts["total_mentions"]

# Filtrer les orateurs avec au moins 10 mentions True
filtered = counts[counts["true_mentions"] >= 10]

# Trier par proportion décroissante et garder les 40 premiers
top40 = filtered.sort_values("proportion_true", ascending=False).head(40).reset_index(drop=True)

top40

,nom_orateur,total_mentions,true_mentions,proportion_true
0,Mme Sonia Backès,182,36,0.197802
1,M. Éric Poulliat,407,67,0.164619
2,M. Philippe Gomès,214,31,0.144860
3,M. Pierre-Yves Bournazel,452,61,0.134956
4,M. Yannick Kerlogot,93,12,0.129032
5,M. Roger Chudeau,212,26,0.122642
6,M. Gabriel Serville,398,47,0.118090
7,Mme Catherine Kamowski,211,24,0.113744
8,M. Guillaume Larrivé,660,74,0.112121
9,M. Louis Aliot,90,10,0.111111


In [50]:
# Compter le nombre de fois où chaque orateur dit "République"
counts = (
    df.groupby("nom_orateur")["Republique_FDM_Regex"]
    .agg(total_mentions="count", true_mentions=lambda x: (x == True).sum())
    .reset_index()
)

# Calcul du nombre de True/False par orateur
df_orateurs = (
    df.groupby("nom_orateur")["Republique_FDM_Regex"]
    .value_counts(normalize=False)  # comptage
    .unstack(fill_value=0)          # colonnes True / False
)

# Calcul de la proportion de True
df_orateurs["proportion_true"] = df_orateurs[True] / (df_orateurs[True] + df_orateurs[False])

# Filtrer les orateurs avec au moins 10 mentions True
filtered = counts[counts["true_mentions"] >= 10]

# Trier par proportion décroissante et garder les 40 premiers
top40 = filtered.sort_values("proportion_true", ascending=False).head(40).reset_index(drop=True)

KeyError: 'proportion_true'

In [25]:
# Calcul du nombre de True/False par orateur
df_orateurs = (
    df.groupby("nom_orateur")["Republique_FDM_Regex"]
    .value_counts(normalize=False)  # comptage
    .unstack(fill_value=0)          # colonnes True / False
)

# Calcul de la proportion de True
df_orateurs["proportion_true"] = df_orateurs[True] / (df_orateurs[True] + df_orateurs[False])

In [26]:
df_orateurs.sort_values("proportion_true", ascending=False).head(40).reset_index()

Republique_FDM_Regex,nom_orateur,False,True,proportion_true
0,M. Bruno Questel (LaREM),0,1,1.000000
1,Mme Mélanie Vogel,0,1,1.000000
2,M. Rouslan Stefantchouk,0,1,1.000000
3,Mme Julie Delpech (RE),0,1,1.000000
4,Mme Dominique Vérien,0,1,1.000000
5,M. Quentin Bataillon (RE),0,1,1.000000
6,Mme Anne-Laurence Petel (RE),0,1,1.000000
7,M. Alexandre Vincendet (LR),0,1,1.000000
8,Mme Maryse Carrère,0,1,1.000000
9,"M. Vincent Ploquin,",0,1,1.000000


## Pause ici 
problème étant que certains parlent très peu, ou utilisent très peu et ont donc une proportion égale à 0. 
Il faudrait réussir à croiser ceux qui en parlent le plus et les % ou alors fixer un seuil à partir duquel on regarde les proportions (genre plus de 10 ?)

In [ ]:
# Calcul du nombre de True/False par orateur
df_orateurs = (
    df.groupby("nom_orateur")["Republique_FDM_Regex"]
    .value_counts(normalize=False)  # comptage
    .unstack(fill_value=0)          # colonnes True / False
)

# Calcul de la proportion de True
df_orateurs["proportion_true"] = df_orateurs[True] / (df_orateurs[True] + df_orateurs[False])

# Trier et prendre les 20 orateurs avec la proportion la plus élevée
df_top20 = df_orateurs.sort_values("proportion_true", ascending=False).head(20).reset_index()

# Graphique
fig_top20 = px.bar(
    df_top20,
    x="nom_orateur",
    y="proportion_true",
    title="Top 20 orateurs par proportion de mentions de 'République'",
    labels={"nom_orateur": "Personnel politique", "proportion_true": "% 'République'"},
    template="plotly_white",
)

fig_top20.update_layout(xaxis_tickangle=-45)

fig_top20.show()


In [14]:
df_proportion = (
    df.groupby(df["dateSeance_day"].dt.date)["Republique_FDM_Regex"]
    .value_counts(normalize=True)  # calcule directement les proportions
    .rename("proportion")
    .reset_index()
)
df_proportion["nom_orateur"].value_counts()

KeyError: 'nom_orateur'

In [51]:
fig_top_orateurs = px.bar(
    df["nom_orateur"].value_counts()[0:10],
    # x=top_counts.index,
    # y=top_counts.values,
    labels={"value": "Nombre d'occurence", "Personnel politique": "Orateur"},
    title="Top 20 personnel politique par occurence de la République",
    template="plotly_white",
)
fig_top_orateurs.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
)
fig_top_orateurs.show()

In [53]:
df_16e = df [df["legislature"]== 16]
df_15e = df [df["legislature"]== 15]

In [55]:
df_16e["nom_orateur"].value_counts()[0:20]

nom_orateur
M. Benjamin Lucas          167
M. Gérald Darmanin         166
Mme Élisabeth Borne        130
M. Antoine Léaument        109
M. Gabriel Attal            63
M. Éric Dupond-Moretti      61
Mme Mathilde Panot          54
M. Sébastien Lecornu        43
Mme Aurore Bergé            38
Mme Sonia Backès            36
M. Boris Vallaud            35
M. Jean-Philippe Tanguy     34
M. Arthur Delaporte         33
M. Sébastien Jumel          32
M. Nicolas Sansu            31
Mme Cyrielle Chatelain      30
M. Pierre Dharréville       30
M. Alexis Corbière          30
M. Mathieu Lefèvre          30
M. Jérôme Guedj             28
Name: count, dtype: int64

In [54]:
df_15e["nom_orateur"].value_counts()[0:20]


nom_orateur
M. Gérald Darmanin            306
M. Alexis Corbière            242
M. Jean-Luc Mélenchon         223
M. Sébastien Jumel            203
M. Jean-Michel Blanquer       180
M. Éric Coquerel              155
M. Stéphane Peu               154
M. Éric Ciotti                154
Mme Danièle Obono             133
M. Ugo Bernalicis             130
Mme Marlène Schiappa          128
M. Philippe Gosselin          126
M. Jean-Christophe Lagarde    114
Mme Mathilde Panot            112
M. Dominique Potier           102
M. Pierre Dharréville         100
Mme Nicole Belloubet           99
M. Bastien Lachaud             98
M. Edouard Philippe            89
M. Raphaël Schellenberger      87
Name: count, dtype: int64

In [58]:
# 16e législature 
fig_top_orateurs = px.bar(
    df_16e["nom_orateur"].value_counts()[0:10],
    # x=top_counts.index,
    # y=top_counts.values,
    labels={"value": "Nombre d'occurence", "Personnel politique": "Orateur"},
    title="Top 20 personnel politique par occurence de la République lors de la 16e législature",
    template="plotly_white",
)
fig_top_orateurs.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
)
fig_top_orateurs.show()

#### Les principaux orateurs par années/semaines/séances

In [60]:
# Par année 
annee = 2021
df_annee = df_annee[df_annee["dateSeance_day"].dt.year == annee]

# Recompter les orateurs sur cette période
df_counts = df_annee["nom_orateur"].value_counts().head(10).reset_index()
df_counts.columns = ["nom_orateur", "occurrences"]

# Graphique
fig_top_orateurs_annee = px.bar(
    df_counts,
    x="nom_orateur",
    y="occurrences",
    title=f"Top 10 orateurs en {annee}",
    template="plotly_white",
)

fig_top_orateurs_annee.update_layout(xaxis_tickangle=-45, showlegend=False)
fig_top_orateurs_annee.show()


#### Regarder plus en détail les jours significatifs

In [85]:
# Par jour

df_jour = df[["dateSeanceJour"] == "vendredi 05 février 2021"]

KeyError: False

In [ ]:
df_jour

In [82]:
# Par jour
jour = "vendredi 5 février 2021"
df_jour = df[["dateSeanceJour"] == jour]

# Recompter les orateurs sur cette période
df_counts = df_jour["nom_orateur"].value_counts().head(10).reset_index()
df_counts.columns = ["nom_orateur", "occurrences"]

# Graphique
fig_top_orateurs_jour = px.bar(
    df_counts,
    x="nom_orateur",
    y="occurrences",
    title=f"Top 10 orateurs en {jour}",
    template="plotly_white",
)

fig_top_orateurs_jour.update_layout(xaxis_tickangle=-45, showlegend=False)
fig_top_orateurs_jour.show()


KeyError: False

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Top 10 orateurs
top_orateurs = df["nom_orateur"].value_counts().index[:10].tolist()

# Grouper par semaine et orateur
df_orateur = df[df["nom_orateur"].isin(top_orateurs)]
df_grouped_orateur = (
    df_orateur.groupby([pd.Grouper(key="dateSeance_day", freq="MS"), "nom_orateur"])
    .size()
    .reset_index(name="mentions")
)

cols = 4
rows = (len(top_orateurs) + cols - 1) // cols
max_y = df_grouped_orateur["mentions"].max()

fig_orateurs_time = make_subplots(
    rows=rows, cols=cols, shared_xaxes=True, subplot_titles=top_orateurs
)

for idx, orateur in enumerate(top_orateurs):
    row = idx // cols + 1
    col = idx % cols + 1
    data_orateur = df_grouped_orateur[df_grouped_orateur["nom_orateur"] == orateur]
    fig_orateurs_time.add_trace(
        go.Bar(
            x=data_orateur["dateSeance_day"], y=data_orateur["mentions"], name=orateur
        ),
        row=row,
        col=col,
    )

fig_orateurs_time.update_layout(
    height=300 * rows,
    width=1200,
    title_text="Dynamique temporelle des mentions par orateur",
    showlegend=False,
    template="plotly_white",
)

for row in range(1, rows + 1):
    for col in range(1, cols + 1):
        fig_orateurs_time.update_yaxes(range=[0, max_y], row=row, col=col)

for col in range(1, cols + 1):
    fig_orateurs_time.update_xaxes(title_text="Date", row=rows, col=col)

for row in range(1, rows + 1):
    fig_orateurs_time.update_yaxes(title_text="Nombre de mentions", row=row, col=1)

fig_orateurs_time.show()


In [ ]:
# Pour la 15e législature 

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Top 10 orateurs
top_orateurs = df_15e["nom_orateur"].value_counts().index[:4].tolist()

# Grouper par semaine et orateur
df_orateur_15e = df_15e[df_15e["nom_orateur"].isin(top_orateurs)]
df_grouped_orateur_15e = (
    df_orateur_15e.groupby([pd.Grouper(key="dateSeance_day", freq="MS"), "nom_orateur"])
    .size()
    .reset_index(name="mentions")
)

cols = 4
rows = (len(top_orateurs) + cols - 1) // cols
max_y = df_grouped_orateur_15e["mentions"].max()

fig_orateurs_time = make_subplots(
    rows=rows, cols=cols, shared_xaxes=True, subplot_titles=top_orateurs
)

for idx, orateur in enumerate(top_orateurs):
    row = idx // cols + 1
    col = idx % cols + 1
    data_orateur = df_grouped_orateur_15e[df_grouped_orateur_15e["nom_orateur"] == orateur]
    fig_orateurs_time.add_trace(
        go.Bar(
            x=data_orateur["dateSeance_day"], y=data_orateur["mentions"], name=orateur
        ),
        row=row,
        col=col,
    )

fig_orateurs_time.update_layout(
    height=300 * rows,
    width=1200,
    title_text="Dynamique temporelle des mentions par orateur",
    showlegend=False,
    template="plotly_white",
)

for row in range(1, rows + 1):
    for col in range(1, cols + 1):
        fig_orateurs_time.update_yaxes(range=[0, max_y], row=row, col=col)

for col in range(1, cols + 1):
    fig_orateurs_time.update_xaxes(title_text="Date", row=rows, col=col)

for row in range(1, rows + 1):
    fig_orateurs_time.update_yaxes(title_text="Nombre de mentions", row=row, col=1)

fig_orateurs_time.show()


### Genre (nécessite d'avoir fait la fusion)

In [ ]:
df["civ"] = df["civ"].replace({"M.": "Homme", "Mme": "Femme"})

In [ ]:
fig = px.bar(df["civ"].value_counts())
fig.update_layout(
    title="Répartition des genres (civ)", template="plotly_white", showlegend=False
)
fig.show()